In [2]:

%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import BatchSampler, SequentialSampler
try:
    import cPickle as thepickle
except ImportError:
    import _pickle as thepickle

import gzip
import numpy as np
torch.set_default_tensor_type(torch.DoubleTensor)

In [3]:
from DeepCCA import DeepCCA , Model

In [14]:
device = torch.device('cpu')
print("Using", torch.cuda.device_count(), "GPUs")

# the path to save the final learned features
save_to = './new_features.gz'

# the size of the new space learned by the model (number of the new features)
outdim_size = 10

# size of the input for view 1 and view 2
input_shape1 = 6
input_shape2 = 47

# number of layers with nodes in each one
layer_sizes1 = [1024, 1024, 1024, outdim_size]
layer_sizes2 = [1024, 1024, 1024, outdim_size]

# the parameters for training the network
learning_rate = 1e-3
epoch_num = 10
batch_size = 800
# apply linear CCA on model output
linear_cca= False

# the regularization parameter of the network
# seems necessary to avoid the gradient exploding especially when non-saturating activations are used
reg_par = 1e-5

# specifies if all the singular values should get used to calculate the correlation or just the top outdim_size ones
# if one option does not work for a network or dataset, try the other one
use_all_singular_values = False

# if a linear CCA should get applied on the learned features extracted from the networks
# it does not affect the performance on noisy MNIST significantly
apply_linear_cca = True
# end of parameters section
############

Using 0 GPUs


In [10]:
X1 = np.loadtxt("/content/mfeat-mor")
X2 = np.loadtxt("/content/mfeat-zer")

X1 = torch.from_numpy(X1)
X2 = torch.from_numpy(X2)



In [15]:
 model = Model(layer_sizes1, layer_sizes2, input_shape1,input_shape2, outdim_size, use_all_singular_values).double()
 deepcca = DeepCCA(model,  outdim_size , epoch_num , batch_size , learning_rate , reg_par , linear_cca  )


[ INFO : 2020-12-13 17:18:03,071 ] - DataParallel(
  (module): Model(
    (model1): MlpNet(
      (layers): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=6, out_features=1024, bias=True)
          (1): Sigmoid()
          (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        )
        (1): Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
          (1): Sigmoid()
          (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        )
        (2): Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
          (1): Sigmoid()
          (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        )
        (3): Sequential(
          (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
          (1): Linear(in_features=1024, out_features=10, bias=True)
        

In [16]:
deepcca.fit(X1 , X2)

[ INFO : 2020-12-13 17:18:10,163 ] - Epoch 1/10 - time: 3.26 - training_loss: -2.9546
[ INFO : 2020-12-13 17:18:13,435 ] - Epoch 2/10 - time: 3.27 - training_loss: -3.4379
[ INFO : 2020-12-13 17:18:16,687 ] - Epoch 3/10 - time: 3.25 - training_loss: -3.6870
[ INFO : 2020-12-13 17:18:19,949 ] - Epoch 4/10 - time: 3.26 - training_loss: -3.8271
[ INFO : 2020-12-13 17:18:23,212 ] - Epoch 5/10 - time: 3.26 - training_loss: -3.9346
[ INFO : 2020-12-13 17:18:26,487 ] - Epoch 6/10 - time: 3.27 - training_loss: -4.0597
[ INFO : 2020-12-13 17:18:29,753 ] - Epoch 7/10 - time: 3.26 - training_loss: -4.1702
[ INFO : 2020-12-13 17:18:32,985 ] - Epoch 8/10 - time: 3.23 - training_loss: -4.2569
[ INFO : 2020-12-13 17:18:36,233 ] - Epoch 9/10 - time: 3.25 - training_loss: -4.3769
[ INFO : 2020-12-13 17:18:39,492 ] - Epoch 10/10 - time: 3.26 - training_loss: -4.4586


In [17]:
outputs = deepcca.transform(X1,X2, linear_cca)
